# Chapter 4: Sample Sizes for Budgeting

Converted from R to Python

## Introduction

There are two types of sampling in interim tests:

1. **Discovery sampling** for discovery of *out-of-control* transaction streams
2. **Attribute sampling** for estimating transaction error rate

Discovery sampling sets a sample size that is likely to discover at least one error in the sample if the actual transaction error rate exceeds the minimum acceptable error-rate (alternatively called the *out-of-control* rate of error).

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import auditanalytics as aa

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Discovery Sampling

Let's visualize how sample size changes with confidence level:

In [ ]:
# Create a range of confidence levels
confidence_levels = np.arange(0.70, 0.99, 0.01)

# Calculate sample sizes for each confidence level
sample_sizes = [aa.core.discovery_sample_size(conf, 0.05) for conf in confidence_levels]

# Plot the relationship
plt.figure(figsize=(10, 6))
plt.plot(confidence_levels, sample_sizes, linewidth=2)
plt.xlabel('Confidence Level', fontsize=12)
plt.ylabel('Sample Size', fontsize=12)
plt.title('Discovery Sample Size vs. Confidence Level (5% Error Rate)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

### Example: 95% Confidence, 5% Intolerable Error Rate

In [ ]:
confidence = 0.95
intolerable_rate = 0.05
n = aa.core.discovery_sample_size(confidence, intolerable_rate)

print(f"Discovery sample size = {n}")
print(f"\nInterpretation: With a sample of {n} transactions, we have {confidence*100}% confidence")
print(f"that we will detect at least one error if the error rate is {intolerable_rate*100}% or higher.")

## Attribute Sampling

Attribute sampling is used when discovery sampling suggests a transaction stream is out of control. It helps estimate the actual error rate.

### Attribute Sampling for Rate of Errors

In [ ]:
# Attribute sample for estimating 'rate' of errors
size = 1000                    # number of transactions
delta = 0.05 * size           # detect 5% occurrence error
sigma = 0.3 * size            # variability (guess ~1/3)

result = aa.core.attribute_sample_size(
    size=size,
    delta_rate=0.05,
    sigma_rate=sigma
)

print(f"Attribute sample size for occurrence of error = {result['occurrence']}")

### Attribute Sampling for Amount of Errors

In [ ]:
# Attribute sample for estimating 'amount' of errors
total_amount = 100000         # total amount of transactions
mu = 50                       # average value of transaction
delta = 0.05 * mu             # detect 5% amount intolerable error
sigma = 30                    # variability

n_amount = aa.core.attribute_sample_size_amount(
    total_amount=total_amount,
    mean_transaction=mu,
    delta_rate=0.05,
    sigma=sigma
)

print(f"Attribute sample size for amount of error = {n_amount}")

## Acceptance Sampling for Substantive Testing

At year-end, substantive samples are focused on acceptance sampling to determine if the account balance is 'fairly stated' (does not contain material error).

In [ ]:
# Acceptance sample for estimating 'amount' of error in an account balance
account_balance = 100000      # Account balance
mu = 50                       # average value of account transaction
delta = 0.05 * mu             # detect 5% amount
sigma = 30                    # variability

n_acceptance = aa.core.acceptance_sample_size(
    account_balance=account_balance,
    mean_transaction=mu,
    delta_rate=0.05,
    sigma=sigma
)

print(f"Acceptance sample size for amount of error = {n_acceptance}")

## Comparison of Sample Sizes

Let's compare the different sampling approaches:

In [ ]:
# Create a comparison table
comparison = pd.DataFrame({
    'Sampling Type': ['Discovery', 'Attribute (Rate)', 'Attribute (Amount)', 'Acceptance'],
    'Sample Size': [n, result['occurrence'], n_amount, n_acceptance],
    'Purpose': [
        'Detect if error rate > 5%',
        'Estimate error occurrence rate',
        'Estimate monetary error',
        'Test if account is fairly stated'
    ]
})

print("\nSample Size Comparison:")
print(comparison.to_string(index=False))

## Practical Application: Accounts Receivable

Consider sampling sales invoices from the accounts receivable aging report. We need to:

1. Compare invoices to supporting documentation
2. Verify correct amounts, customers, and dates
3. Match invoice dates to shipment dates
4. Examine invoices from subsequent periods

In [ ]:
# Example: A/R with turnover consideration
sales_account = 1000000       # $1M in sales
ar_turnover = 10              # 10 times per year
error_rate = 0.05             # 5% error rate found in interim testing

# Impact on year-end accounts
sales_error = sales_account * error_rate
ar_error = sales_error / ar_turnover

print(f"\nImpact of 5% error rate:")
print(f"  Sales misstatement: ${sales_error:,.0f}")
print(f"  A/R misstatement: ${ar_error:,.0f}")
print(f"\nWith $10,000 materiality:")
print(f"  Sales fairly stated? {sales_error < 10000}")
print(f"  A/R fairly stated? {ar_error < 10000}")

## Conclusion

This notebook demonstrates the three main types of statistical sampling in auditing:

1. **Discovery Sampling** - Detects whether systems are out of control
2. **Attribute Sampling** - Estimates the rate and amount of errors
3. **Acceptance Sampling** - Tests whether accounts are fairly stated

All formulas are based on statistical power analysis and provide scientifically defensible sample sizes.